# **Cantoría Evaluation**

Developed by André Paiva

Based on Cantoría Dataset and papers by Helena Cuesta and Emilia Gómez

In [ ]:
import sys
import math
import numpy as np
import pandas as pd
import sscs

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
SONGLIST = ['CEA', 'EJB1', 'EJB2', 'HCB', 'LBM1', 'LBM2', 'LJT1', 'LJT2', 'LNG', 'RRC', 'SSS', 'THM', 'VBP', 'YSM']

ckpt_dir = './Checkpoints/mask_voas_v2_treino1.keras'
model = sscs.mask_voas_cnn_v2_model()
sscs.load_weights(model, ckpt_dir=ckpt_dir)

s_f_scores = np.zeros((1))
a_f_scores = np.zeros((1))
t_f_scores = np.zeros((1))
b_f_scores = np.zeros((1))

for song in SONGLIST:
    df_audio = pd.read_hdf('Datasets/Cantoria/Audio/Cantoria_' + song + '_Mix.h5', key='mix', mode='r')
    df_pyin_s = pd.read_csv('Datasets/Cantoria/F0_crepe/Cantoria_' + song + '_S.csv', header=None, engine='pyarrow')
    df_pyin_a = pd.read_csv('Datasets/Cantoria/F0_crepe/Cantoria_' + song + '_A.csv', header=None, engine='pyarrow')
    df_pyin_t = pd.read_csv('Datasets/Cantoria/F0_crepe/Cantoria_' + song + '_T.csv', header=None, engine='pyarrow')
    df_pyin_b = pd.read_csv('Datasets/Cantoria/F0_crepe/Cantoria_' + song + '_B.csv', header=None, engine='pyarrow')

    mix = df_audio.to_numpy().T
    ground_timescale = df_pyin_s[0].to_numpy()

    s_true = np.array([freq/1000 if freq > 100000 else freq for freq in df_pyin_s[1].to_numpy()])
    a_true = np.array([freq/1000 if freq > 100000 else freq for freq in df_pyin_a[1].to_numpy()])
    t_true = np.array([freq/1000 if freq > 100000 else freq for freq in df_pyin_t[1].to_numpy()])
    b_true = np.array([freq/1000 if freq > 100000 else freq for freq in df_pyin_b[1].to_numpy()])

    s_true = np.array([32.7 if freq == 0 else freq for freq in s_true])
    a_true = np.array([32.7 if freq == 0 else freq for freq in a_true])
    t_true = np.array([32.7 if freq == 0 else freq for freq in t_true])
    b_true = np.array([32.7 if freq == 0 else freq for freq in b_true])

    splits = mix.shape[1]//256
    splits_diff = 256 - (mix.shape[1] - splits * 256)
    fill = np.zeros((360, splits_diff))
    mix_filled = np.concatenate((np.copy(mix), fill), axis=1)
    mix_filled = np.reshape(mix_filled, (360, -1, 256, 1)).transpose((1, 0, 2, 3))
    batches = math.ceil(mix_filled.shape[0]/24)

    s_pred_result = np.zeros((0, 360, 256))
    a_pred_result = np.zeros((0, 360, 256))
    t_pred_result = np.zeros((0, 360, 256))
    b_pred_result = np.zeros((0, 360, 256))

    for i in range(batches):
        s_pred, a_pred, t_pred, b_pred = model.predict(mix_filled[i*24:(i+1)*24])
        s_pred_result = np.append(s_pred_result, s_pred, axis=0)
        a_pred_result = np.append(a_pred_result, a_pred, axis=0)
        t_pred_result = np.append(t_pred_result, t_pred, axis=0)
        b_pred_result = np.append(b_pred_result, b_pred, axis=0)

    s_pred_result = sscs.prediction_postproc(s_pred_result)[:, :mix.shape[1]]
    a_pred_result = sscs.prediction_postproc(a_pred_result)[:, :mix.shape[1]]
    t_pred_result = sscs.prediction_postproc(t_pred_result)[:, :mix.shape[1]]
    b_pred_result = sscs.prediction_postproc(b_pred_result)[:, :mix.shape[1]]

    s_f_score = sscs.f_score_time_freq(s_true, s_pred_result, ground_timescale, true_bin=False)
    a_f_score = sscs.f_score_time_freq(a_true, a_pred_result, ground_timescale, true_bin=False)
    t_f_score = sscs.f_score_time_freq(t_true, t_pred_result, ground_timescale, true_bin=False)
    b_f_score = sscs.f_score_time_freq(b_true, b_pred_result, ground_timescale, true_bin=False)

    s_f_scores = np.append(s_f_scores, [s_f_score], axis=0)
    a_f_scores = np.append(a_f_scores, [a_f_score], axis=0)
    t_f_scores = np.append(t_f_scores, [t_f_score], axis=0)
    b_f_scores = np.append(b_f_scores, [b_f_score], axis=0)

    sscs.song_to_midi(s_pred_result, a_pred_result, t_pred_result, b_pred_result, './MIDI/' + song + '.mid')
    
s_f_scores = s_f_scores[1:]
a_f_scores = a_f_scores[1:]
t_f_scores = t_f_scores[1:]
b_f_scores = b_f_scores[1:]
    

In [3]:
s_metrics = np.array([np.mean(s_f_scores), np.median(s_f_scores), np.std(s_f_scores)])
a_metrics = np.array([np.mean(a_f_scores), np.median(a_f_scores), np.std(a_f_scores)])
t_metrics = np.array([np.mean(t_f_scores), np.median(t_f_scores), np.std(t_f_scores)])
b_metrics = np.array([np.mean(b_f_scores), np.median(b_f_scores), np.std(b_f_scores)])

df_metrics = pd.DataFrame(columns=['Metrics', 'S', 'A', 'T', 'B'])
df_metrics['Metrics'] = ['Mean', 'Median', 'Std. Deviation']
df_metrics['S'] = s_metrics
df_metrics['A'] = a_metrics
df_metrics['T'] = t_metrics
df_metrics['B'] = b_metrics

df_metrics


,Metrics,S,A,T,B
0,Mean,0.722948,0.632969,0.627644,0.686482
1,Median,0.748152,0.642596,0.636483,0.695847
2,Std. Deviation,0.105555,0.117296,0.117348,0.091088
